# SK-6-ProcessFramework : Workflows et Orchestration

**Navigation** : [<< 05-VectorStores](05-SemanticKernel-VectorStores.ipynb) | [Index](README.md) | [07-MultiModal >>](07-SemanticKernel-MultiModal.ipynb)

---

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Comprendre la difference entre **Plugins**, **Agents** et **Processes**
2. Creer des **Steps** (etapes) de workflow
3. Orchestrer des etapes avec **ProcessBuilder**
4. Gerer l'**etat** entre les etapes
5. Implementer des **conditions** et **branchements**

### Prerequis

- Python 3.10+
- Notebooks 01-05 completes
- Cle API OpenAI configuree (`.env`)

### Duree estimee : 40 minutes

---

## Sommaire

| Section | Contenu | Concepts cles |
|---------|---------|---------------|
| 1 | Introduction | Plugins vs Agents vs Processes |
| 2 | Architecture | Steps, State, Events |
| 3 | ProcessBuilder | Creation de workflow |
| 4 | Gestion d'etat | Passage de donnees |
| 5 | Conditions | Branchements dynamiques |
| 6 | Cas d'usage | Validation de contenu |
| 7 | Conclusion | Resume, exercices |

> **Process Framework** : Introduit en preview dans SK 1.x, GA prevu Q2 2026. Ce framework permet d'orchestrer des workflows complexes avec gestion d'etat, checkpoints, et human-in-the-loop. Ideal pour les pipelines multi-etapes.

In [1]:
# Installation
%pip install semantic-kernel python-dotenv --quiet

import os
from dotenv import load_dotenv
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAIChatPromptExecutionSettings

# Chargement du fichier .env
load_dotenv("../.env")

kernel = Kernel()
kernel.add_service(OpenAIChatCompletion(service_id="default"))

# Settings par defaut pour les appels chat
default_settings = OpenAIChatPromptExecutionSettings()

print("Process Framework - Demo conceptuelle")


[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: C:\Users\jsboi\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


Process Framework - Demo conceptuelle


### Interprétation : Configuration du kernel

**Sortie obtenue** : Kernel SK configuré avec service OpenAI

| Aspect | Valeur | Signification |
|--------|--------|---------------|
| **Service** | OpenAIChatCompletion | Modèle de chat (GPT-3.5/4) |
| **Service ID** | "default" | Identifiant pour multi-services |
| **Settings** | OpenAIChatPromptExecutionSettings | Configuration réutilisable |

**Points clés** :
1. Le kernel est le point d'entrée central de SK
2. `add_service()` permet de configurer plusieurs modèles (OpenAI, Azure, Anthropic)
3. Les `settings` par défaut évitent de répéter la config à chaque appel (requis SK 1.39+)
4. Le service_id permet de gérer plusieurs modèles simultanément

**Note technique** : SK 1.39+ rend les `settings` obligatoires pour `get_chat_message_contents()`. Avant, ils étaient optionnels.

## 1. Introduction : Plugins vs Agents vs Processes

SK offre trois niveaux d'abstraction pour structurer votre code IA :

| Niveau | Abstraction | Granularite | Exemple |
|--------|-------------|-------------|---------|
| **Plugin** | Fonction | Atomique | `summarize_text()` |
| **Agent** | Entite autonome | Tache | "Resume ce document" |
| **Process** | Workflow | Pipeline | Generate -> Review -> Publish |

### Quand utiliser quoi ?

```
Complexite croissante --->

┌─────────────┐    ┌─────────────┐    ┌─────────────┐
│   PLUGIN    │    │   AGENT     │    │  PROCESS    │
│             │    │             │    │             │
│ - Fonction  │    │ - Autonome  │    │ - Pipeline  │
│ - Stateless │    │ - Stateful  │    │ - Multi-step│
│ - Simple    │    │ - Tools     │    │ - Checkpts  │
└─────────────┘    └─────────────┘    └─────────────┘
    "call()"         "invoke()"        "run()"
```

**Indicateurs pour utiliser un Process** :
- Pipeline avec plusieurs etapes sequentielles/paralleles
- Besoin de checkpoints (reprise apres echec)
- Human-in-the-loop (approbation humaine)
- Etat partage entre etapes
- Conditions de branchement complexes

## 2. Architecture du Process Framework

### Composants principaux

```
┌─────────────────────────────────────────────────────┐
│                    PROCESS                          │
│  ┌─────────────────────────────────────────────┐   │
│  │                 STATE                        │   │
│  │   (donnees partagees entre etapes)          │   │
│  └─────────────────────────────────────────────┘   │
│                                                     │
│  ┌─────────┐     ┌─────────┐     ┌─────────┐      │
│  │  STEP   │────>│  STEP   │────>│  STEP   │      │
│  │ Generate│     │ Review  │     │ Publish │      │
│  └─────────┘     └─────────┘     └─────────┘      │
│       ↓              ↓               ↓             │
│  ┌─────────────────────────────────────────────┐   │
│  │               EVENTS                         │   │
│  │  (declencheurs entre etapes)                │   │
│  └─────────────────────────────────────────────┘   │
└─────────────────────────────────────────────────────┘
```

| Composant | Role | Exemple |
|-----------|------|---------|
| **Process** | Conteneur du workflow | ContentValidationProcess |
| **Step** | Etape d'execution | GenerateStep, ReviewStep |
| **State** | Donnees partagees | `{"content": "...", "approved": False}` |
| **Event** | Declencheur | OnContentGenerated, OnReviewComplete |

## 3. Creation de Steps

Les Steps sont les briques de base des processes.

In [2]:
from dataclasses import dataclass
from typing import Optional
from semantic_kernel.contents import ChatHistory

# Definition de l'etat du process
@dataclass
class ContentState:
    """Etat partage entre les etapes du workflow."""
    topic: str
    draft_content: Optional[str] = None
    review_feedback: Optional[str] = None
    is_approved: bool = False
    final_content: Optional[str] = None

# Step 1: Generation de contenu
async def generate_content_step(state: ContentState, kernel: Kernel) -> ContentState:
    """Genere un brouillon de contenu."""
    print(f"[Step 1] Generation du contenu sur: {state.topic}")
    
    chat_service = kernel.get_service(service_id="default")
    history = ChatHistory()
    history.add_user_message(f"Ecris un paragraphe court sur: {state.topic}")
    
    # SK 1.39+: settings requis
    response = await chat_service.get_chat_message_contents(
        chat_history=history,
        settings=default_settings
    )
    state.draft_content = str(response[0])
    
    print(f"[Step 1] Brouillon genere: {len(state.draft_content)} caracteres")
    return state

# Step 2: Review du contenu
async def review_content_step(state: ContentState, kernel: Kernel) -> ContentState:
    """Evalue la qualite du contenu."""
    print(f"[Step 2] Review du contenu")
    
    chat_service = kernel.get_service(service_id="default")
    history = ChatHistory()
    history.add_user_message(
        f"""Evalue ce contenu et reponds par 'APPROVED' si c'est bon, sinon donne des suggestions:
        
        {state.draft_content}"""
    )
    
    # SK 1.39+: settings requis
    response = await chat_service.get_chat_message_contents(
        chat_history=history,
        settings=default_settings
    )
    state.review_feedback = str(response[0])
    state.is_approved = "approved" in state.review_feedback.lower()
    
    print(f"[Step 2] Approuve: {state.is_approved}")
    return state

# Step 3: Publication
async def publish_content_step(state: ContentState) -> ContentState:
    """Publie le contenu final."""
    print(f"[Step 3] Publication du contenu")
    
    if state.is_approved:
        state.final_content = state.draft_content
        print(f"[Step 3] Contenu publie avec succes!")
    else:
        print(f"[Step 3] Contenu rejete - necessite revision")
    
    return state

print("Steps definis: generate, review, publish")

Steps definis: generate, review, publish


### Interprétation : Définition des steps

**Sortie obtenue** : 3 fonctions asynchrones créées (generate, review, publish)

| Step | Entrée | Sortie | Effet de bord |
|------|--------|--------|---------------|
| **generate_content_step** | state.topic | state.draft_content | Appel LLM |
| **review_content_step** | state.draft_content | state.review_feedback, is_approved | Appel LLM |
| **publish_content_step** | state.is_approved | state.final_content | Logique métier |

**Points clés** :
1. **State pattern** : Chaque step reçoit et retourne l'état complet (`ContentState`)
2. **Immutabilité partielle** : L'objet state est muté in-place (Python dataclass mutable)
3. **Async/await** : Les steps sont `async` pour supporter les appels réseau (LLM)
4. **Separation of concerns** : Chaque step a une responsabilité unique (SRP)

**Note technique** : SK 1.39+ requiert `settings` dans `get_chat_message_contents()`. Les versions précédentes acceptaient `settings=None`.

In [3]:
# Execution manuelle du pipeline
async def run_content_pipeline(topic: str, kernel: Kernel) -> ContentState:
    """Execute le pipeline de validation de contenu."""
    
    # Initialiser l'etat
    state = ContentState(topic=topic)
    
    # Etape 1: Generation
    state = await generate_content_step(state, kernel)
    
    # Etape 2: Review
    state = await review_content_step(state, kernel)
    
    # Etape 3: Publication (conditionnelle)
    state = await publish_content_step(state)
    
    return state

# Test du pipeline
result = await run_content_pipeline("Les avantages de Semantic Kernel", kernel)

print("\n" + "=" * 60)
print("RESULTAT DU PIPELINE")
print("=" * 60)
print(f"Topic: {result.topic}")
print(f"Approuve: {result.is_approved}")
print(f"Contenu final: {'Oui' if result.final_content else 'Non'}")

[Step 1] Generation du contenu sur: Les avantages de Semantic Kernel


[Step 1] Brouillon genere: 1039 caracteres
[Step 2] Review du contenu


[Step 2] Approuve: True
[Step 3] Publication du contenu
[Step 3] Contenu publie avec succes!

RESULTAT DU PIPELINE
Topic: Les avantages de Semantic Kernel
Approuve: True
Contenu final: Oui


### Interprétation : Exécution du pipeline

**Sortie obtenue** : Contenu généré, reviewé et publié avec succès

| Métrique | Valeur | Explication |
|----------|--------|-------------|
| **Caractères générés** | ~1039 | Paragraphe court produit par GPT |
| **Approbation** | True | Review automatique positive |
| **Étapes exécutées** | 3/3 | Pipeline complet sans échec |
| **Temps estimé** | ~5-10s | 2 appels LLM + logique |

**Points clés** :
1. **Pipeline séquentiel** : Les steps s'exécutent dans l'ordre défini
2. **État partagé** : `state` est passé entre les fonctions (pas de base de données)
3. **Execution conditionnelle** : `publish_step` vérifie `is_approved` avant publication
4. **Traçabilité** : Les `print()` permettent d'observer le flux d'exécution

**Pattern observé** : Generate → Review → Publish (workflow classique de production de contenu)

**Note technique** : Ce pipeline est **stateless** entre exécutions. Pour persister l'état (checkpoints), utiliser Dapr State Store ou un stockage externe.

## 4. Process avec iterations

Un pattern courant : iterer jusqu'a ce qu'une condition soit remplie.

In [4]:
async def run_iterative_pipeline(
    topic: str, 
    kernel: Kernel, 
    max_iterations: int = 3
) -> ContentState:
    """Pipeline avec boucle de revision."""
    
    state = ContentState(topic=topic)
    
    for iteration in range(max_iterations):
        print(f"\n--- Iteration {iteration + 1}/{max_iterations} ---")
        
        # Generation (ou revision)
        if iteration == 0:
            state = await generate_content_step(state, kernel)
        else:
            # Reviser en tenant compte du feedback
            state = await revise_content_step(state, kernel)
        
        # Review
        state = await review_content_step(state, kernel)
        
        # Condition de sortie
        if state.is_approved:
            print(f"\n[SUCCESS] Contenu approuve apres {iteration + 1} iteration(s)")
            break
    
    # Publication
    state = await publish_content_step(state)
    
    return state

async def revise_content_step(state: ContentState, kernel: Kernel) -> ContentState:
    """Revise le contenu selon le feedback."""
    print(f"[Step R] Revision du contenu")
    
    chat_service = kernel.get_service(service_id="default")
    history = ChatHistory()
    history.add_user_message(
        f"""Revise ce contenu en tenant compte du feedback:
        
        CONTENU ORIGINAL:
        {state.draft_content}
        
        FEEDBACK:
        {state.review_feedback}
        
        Produis une version amelioree."""
    )
    
    # SK 1.39+: settings requis
    response = await chat_service.get_chat_message_contents(
        chat_history=history,
        settings=default_settings
    )
    state.draft_content = str(response[0])
    
    print(f"[Step R] Contenu revise")
    return state

# Test du pipeline iteratif
result = await run_iterative_pipeline("L'importance du RAG en IA", kernel, max_iterations=3)

print("\n" + "=" * 60)
print(f"Resultat final: {'PUBLIE' if result.final_content else 'NON PUBLIE'}")


--- Iteration 1/3 ---
[Step 1] Generation du contenu sur: L'importance du RAG en IA


[Step 1] Brouillon genere: 822 caracteres
[Step 2] Review du contenu


[Step 2] Approuve: True

[SUCCESS] Contenu approuve apres 1 iteration(s)
[Step 3] Publication du contenu
[Step 3] Contenu publie avec succes!

Resultat final: PUBLIE


### Interprétation : Pipeline itératif avec révision

**Sortie obtenue** : Contenu approuvé après 1 itération

| Métrique | Valeur | Signification |
|----------|--------|---------------|
| **Itérations max** | 3 | Budget de révisions autorisées |
| **Itérations réelles** | 1 | Contenu approuvé du premier coup |
| **Steps exécutés** | Generate → Review → Publish | Pas besoin de révision |
| **Appels LLM** | 2 | Generate + Review (pas de Revise) |

**Points clés** :
1. **Boucle de feedback** : Le pipeline peut itérer jusqu'à approbation
2. **Early exit** : `break` dès que `is_approved == True` (économise des appels LLM)
3. **Step conditionnel** : `revise_content_step` n'est appelé qu'à partir de l'itération 2
4. **Budget de tokens** : Limiter `max_iterations` pour éviter les coûts incontrôlés

**Pattern observé** : **Critic Loop** (Generate ↔ Revise ↔ Review)

**Cas d'usage réels** :
- Code review automatique (Developer ↔ Reviewer)
- Traduction itérative (Translate ↔ Proofread)
- Extraction de données structurées (Extract ↔ Validate)

**Note technique** : Pour des itérations complexes, LangGraph offre des primitives dédiées (cycles, checkpoints). SK Process est plus adapté aux workflows linéaires avec branchements simples.

## 5. Pattern avec Human-in-the-Loop (conceptuel)

Le Process Framework permet d'integrer des points d'approbation humaine.

In [5]:
# Simulation d'approbation humaine
async def human_approval_step(state: ContentState) -> ContentState:
    """Demande une approbation humaine (simule)."""
    print(f"\n[HUMAN] Contenu a approuver:")
    print("-" * 40)
    print(state.draft_content[:200] + "..." if len(state.draft_content) > 200 else state.draft_content)
    print("-" * 40)
    
    # En production, ceci serait un webhook, email, ou UI
    # Ici on simule une approbation automatique
    print("[HUMAN] Approbation simulee...")
    state.is_approved = True
    
    return state

# Pipeline avec approbation humaine
async def run_human_in_loop_pipeline(topic: str, kernel: Kernel) -> ContentState:
    """Pipeline avec point d'approbation humaine."""
    
    state = ContentState(topic=topic)
    
    # Generation
    state = await generate_content_step(state, kernel)
    
    # Review automatique
    state = await review_content_step(state, kernel)
    
    # Point d'arret pour approbation humaine
    if not state.is_approved:
        print("\n[PROCESS] Review automatique negative - demande approbation humaine")
        state = await human_approval_step(state)
    
    # Publication
    state = await publish_content_step(state)
    
    return state

# Test
result = await run_human_in_loop_pipeline("Les limites des LLMs", kernel)
print(f"\nContenu final publie: {'Oui' if result.final_content else 'Non'}")

[Step 1] Generation du contenu sur: Les limites des LLMs


[Step 1] Brouillon genere: 860 caracteres
[Step 2] Review du contenu


[Step 2] Approuve: True
[Step 3] Publication du contenu
[Step 3] Contenu publie avec succes!

Contenu final publie: Oui


### Interprétation : Human-in-the-Loop

**Sortie obtenue** : Pipeline avec point d'approbation humaine (simulé)

| Composant | Implémentation | Équivalent production |
|-----------|----------------|----------------------|
| **human_approval_step** | Fonction simulée | Webhook, email, UI dashboard |
| **Trigger** | `if not state.is_approved` | Après échec de review auto |
| **Timeout** | Aucun (simulation) | Délai d'attente configurable |
| **Notification** | Print | Slack, Teams, email |

**Points clés** :
1. **Breakpoint asynchrone** : Le process attend une action externe
2. **Deux niveaux de validation** : Review auto (LLM) + Review humaine (fallback)
3. **Stateful** : L'état doit être persisté si le workflow attend des heures/jours
4. **Reprise après pause** : Nécessite un système de checkpoints (Dapr, base de données)

**Architecture pour production** :

```
┌──────────┐    ┌──────────┐    ┌────────────┐
│ Generate │───>│  Review  │───>│ If not OK  │
└──────────┘    └──────────┘    └─────┬──────┘
                                      │
                                      v
                            ┌─────────────────┐
                            │  Send webhook   │
                            │  Wait for event │ <- Humain approuve
                            └────────┬────────┘
                                     │
                                     v
                            ┌─────────────────┐
                            │    Publish      │
                            └─────────────────┘
```

**Implémentations recommandées** :
- **Dapr Workflow** : Intégré SK, support natif human-in-the-loop
- **Temporal** : Workflows durables, versioning
- **Custom webhook** : API REST + queue (Redis, RabbitMQ)

**Note technique** : SK Process Framework (preview) n'a pas encore de primitive dédiée pour human-in-the-loop. Utiliser Dapr Workflows pour cette fonctionnalité en production.

## 6. Comparaison avec alternatives

| Framework | Forces | Cas d'usage |
|-----------|--------|-------------|
| **SK Process** | Integration SK native, Dapr support | Workflows IA simples |
| **LangGraph** | Graphes complexes, cycles | Multi-agent sophistique |
| **Prefect/Airflow** | Scheduling, monitoring | Data pipelines |
| **Temporal** | Durabilite, versioning | Workflows long-running |
| **AutoGen Flows** | Multi-agent natif | Conversations complexes |

### Quand utiliser SK Process Framework ?

| OUI | NON |
|-----|-----|
| Deja dans l'ecosysteme SK | Besoins de scheduling avance |
| Pipelines lineaires simples | Graphes avec cycles complexes |
| Human-in-the-loop basique | Monitoring entreprise |
| Prototypage rapide | Production critique |

# Conclusion

## Resume des concepts

| Concept | Description | Code cle |
|---------|-------------|----------|
| **Process** | Workflow orchestre | Conteneur de steps |
| **Step** | Etape d'execution | Fonction async |
| **State** | Donnees partagees | Dataclass mutable |
| **Condition** | Branchement | `if state.is_approved:` |
| **Iteration** | Boucle jusqu'a succes | `for i in range(max):` |

## Points cles a retenir

1. **Plugins < Agents < Processes** - Choisir le bon niveau d'abstraction
2. **L'etat lie les etapes** - Dataclass partagee entre steps
3. **Les conditions permettent le branchement** - Logique metier dans l'etat
4. **Human-in-the-loop est essentiel** - Points d'arret pour approbation
5. **Preview API** - Peut evoluer avant GA (Q2 2026)

## Exercices suggeres

1. **Pipeline de traduction** : Translate -> Review -> Proofread -> Publish
2. **Multi-agent process** : Integrer AgentGroupChat dans un step
3. **Persistance** : Sauvegarder l'etat entre les executions

## Pour aller plus loin

| Notebook | Contenu |
|----------|--------|
| [07-MultiModal](07-SemanticKernel-MultiModal.ipynb) | DALL-E, Whisper, Vision |
| [08-MCP](08-SemanticKernel-MCP.ipynb) | Model Context Protocol |

---

**Navigation** : [<< 05-VectorStores](05-SemanticKernel-VectorStores.ipynb) | [Index](README.md) | [07-MultiModal >>](07-SemanticKernel-MultiModal.ipynb)